In [ ]:
from roboticstoolbox.models.DH import Panda
import numpy as np

In [ ]:
panda = Panda()

# Centers of masses of Panda not yet implemented in Robotic Toolbox
r_arr = np.array(
    [
        [3.875e-03, 2.081e-03, 0],
        [-3.141e-03, -2.872e-02, 3.495e-03],
        [2.7518e-02, 3.9252e-02, -6.6502e-02],
        [-5.317e-02, 1.04419e-01, 2.7454e-02],
        [-1.1953e-02, 4.1065e-02, -3.8437e-02],
        [6.0149e-02, -1.4117e-02, -1.0517e-02],
        [1.0517e-02, -4.252e-03, -4.5403e-02],
    ]
)

for ii, link in enumerate(panda.links):
    link.r = r_arr[ii, :]

In [ ]:
def link_2_params(link):
    m = link.m
    mc_x, mc_y, mc_z = link.r
    I_xx = link.I[0,0]
    I_xy = link.I[0,1]
    I_xz = link.I[0,2]
    I_yy = link.I[1,1]
    I_yz = link.I[1,2]
    I_zz = link.I[2,2]
    return np.array([m, mc_x, mc_y, mc_z, I_xx, I_xy, I_yy, I_xz, I_yz, I_zz])

In [ ]:
def link_2_params_xml(link):
    m = link.m
    mc_x, mc_y, mc_z = link.r
    I_xx = link.I[0,0]
    I_xy = link.I[0,1]
    I_xz = link.I[0,2]
    I_yy = link.I[1,1]
    I_yz = link.I[1,2]
    I_zz = link.I[2,2]
    return str(m), f'{mc_x} {mc_y} {mc_z}', str(I_xx), str(I_xy), str(I_xz), str(I_yy), str(I_yz), str(I_zz)

In [ ]:
panda

In [ ]:
for ii, link in enumerate(panda.links):
    print(ii,link_2_params(link))

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
tree = ET.parse('urdf/panda.urdf')
root = tree.getroot()

In [ ]:
'1'.isdigit()

In [ ]:
for ii,child in enumerate(root):
    if child.tag == 'link':
        link_n = child.attrib['name'][-1]
        if link_n.isdigit():
            link_n = int(link_n)
        else:
            link_n = 100
        print(link_n, child.tag, child.attrib)
        for jj,subchild in enumerate(child):
            if subchild.tag == 'inertial':
                print('\t', jj, subchild.tag, subchild.attrib)
                if link_n < 8:
                    m, xyz, I_xx, I_xy, I_xz, I_yy, I_yz, I_zz = link_2_params_xml(panda.links[link_n-1])
                for kk,subsubchild in enumerate(subchild):
                    if subsubchild.tag == 'mass' and link_n < 8:
                        subsubchild.attrib['value'] = m
                    if subsubchild.tag == 'origin' and link_n < 8:
                        subsubchild.attrib['xyz'] = xyz
                    if subsubchild.tag == 'inertia' and link_n < 8:
                        subsubchild.attrib['ixx'] = I_xx
                        subsubchild.attrib['ixy'] = I_xy
                        subsubchild.attrib['ixz'] = I_xz
                        subsubchild.attrib['iyy'] = I_yy
                        subsubchild.attrib['iyz'] = I_yz
                        subsubchild.attrib['izz'] = I_zz
                    print('\t\t', kk, subsubchild.tag, subsubchild.attrib)
                    #if subsubchild.tag == 'mass' and link_n < 8:
                    #    print('next mass: ',m)
                    #if subsubchild.tag == 'origin' and link_n < 8:
                    #    print('next xyz: ',xyz)

In [ ]:
tree.write('urdf/panda2.urdf')